In [1]:
#Fundamental librarys to math and stats process
import numpy as np
import numpy.random as nr
import scipy.stats as ss
import math
#data prepared
import pandas as pd

#ML preprocessi
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import sklearn.model_selection as ms
from sklearn.pipeline import make_pipeline
from sklearn import feature_selection as fs

# ML algorithms models
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor 
from sklearn import linear_model as lm

# ML Evaluations
import sklearn.metrics as sklm
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split

#Ploting
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

In [4]:
df = (pd.read_csv('x.csv'))
y = (pd.read_csv('y.csv'))
y = y['evictions']
df = df.drop(['Unnamed: 0'], axis = 1)
print(df.shape)
print(y.shape)

(2546, 26)
(2546,)


In [5]:
list(df.columns)

['renter_occupied_households',
 'pct_renter_occupied',
 'median_gross_rent',
 'rent_burden',
 'pct_af_am',
 'pct_hispanic',
 'pct_am_ind',
 'pct_multiple',
 'pct_other',
 'poverty_rate',
 'pct_unemployment',
 'pct_uninsured_adults',
 'pct_uninsured_children',
 'pct_adult_obesity',
 'pct_diabetes',
 'heart_disease_mortality_per_100k',
 'pct_female',
 'pct_below_18_years_of_age',
 'pct_adults_less_than_a_high_school_diploma',
 'pct_adults_with_high_school_diploma',
 'pct_adults_with_some_college',
 'birth_rate_per_1k',
 'year',
 'rucc',
 'urban_influence',
 'economic_typology']

In [6]:
df.head()

,renter_occupied_households,pct_renter_occupied,median_gross_rent,rent_burden,pct_af_am,pct_hispanic,pct_am_ind,pct_multiple,pct_other,poverty_rate,...,pct_female,pct_below_18_years_of_age,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,birth_rate_per_1k,year,rucc,urban_influence,economic_typology
0,0.002484,0.025221,0.017875,0.022924,0.057196,0.008249,0.000970,0.013199,0.009978,0.026887,...,0.021099,0.021854,0.028364,0.020639,0.018044,0.021756,b,Urban,Micropolitan adjacent to a large metro area,Nonspecialized
1,0.000438,0.021531,0.014372,0.018178,0.001807,0.004397,0.027541,0.023416,0.000000,0.017373,...,0.020187,0.021854,0.008922,0.021882,0.021505,0.018613,a,Urban,Noncore adjacent to a small metro with town of...,Nonspecialized
2,0.000617,0.014890,0.018653,0.022314,0.004757,0.005514,0.001688,0.023817,0.002007,0.017316,...,0.017886,0.014396,0.025149,0.016698,0.019563,0.013314,b,Rural,Noncore adjacent to a small metro and does not...,Recreation
3,0.006504,0.024944,0.016763,0.021942,0.027915,0.009526,0.141646,0.021209,0.028169,0.037923,...,0.020583,0.022807,0.030221,0.018437,0.018831,0.022130,a,Urban,Micropolitan adjacent to a small metro area,Nonspecialized
4,0.000197,0.011945,0.018570,0.020143,0.000596,0.004201,0.005263,0.013822,0.000000,0.015554,...,0.019314,0.016997,0.007139,0.015209,0.026840,0.010250,a,Counties,Small-in a metro area with fewer than 1 millio...,Federal/State government-dependent


### Encode categorical featured

In [7]:
print(df['rucc'].unique())
Features = df['rucc']
enc = preprocessing.LabelEncoder()
enc.fit(Features)
Features = enc.transform(Features)
print(Features)

['Urban' 'Rural' 'Counties']
[2 2 1 ... 0 2 0]


In [8]:
ohe = preprocessing.OneHotEncoder()
encoded = ohe.fit(Features.reshape(-1,1))
Features = encoded.transform(Features.reshape(-1,1)).toarray()
Features[:10,:]

C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [9]:
def encode_string(cat_feature):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_feature)
    enc_cat_feature = enc.transform(cat_feature)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_feature.reshape(-1,1))
    return encoded.transform(enc_cat_feature.reshape(-1,1)).toarray()
    
categorical_columns= ['economic_typology','year']

for col in categorical_columns:
    temp = encode_string(df[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :])    

(2546, 11)
[[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0.]]


C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want 

In [10]:
numcols= df.dtypes[df.dtypes != "object"].index

In [11]:
list(numcols)

['renter_occupied_households',
 'pct_renter_occupied',
 'median_gross_rent',
 'rent_burden',
 'pct_af_am',
 'pct_hispanic',
 'pct_am_ind',
 'pct_multiple',
 'pct_other',
 'poverty_rate',
 'pct_unemployment',
 'pct_uninsured_adults',
 'pct_uninsured_children',
 'pct_adult_obesity',
 'pct_diabetes',
 'heart_disease_mortality_per_100k',
 'pct_female',
 'pct_below_18_years_of_age',
 'pct_adults_less_than_a_high_school_diploma',
 'pct_adults_with_high_school_diploma',
 'pct_adults_with_some_college',
 'birth_rate_per_1k']

In [14]:
x = np.concatenate([df[numcols],Features],axis=1)

In [15]:
y=np.array(y)

In [16]:
print(x.shape)
print(y.shape)

(2546, 33)
(2546,)


In [17]:
# splt train test
nr.seed(9988)
x_train, x_test, y_train, y_test = train_test_split(x,y)

## Train models selected

In [18]:
Boosting_mod = GradientBoostingRegressor (learning_rate=0.1)
Forest_mod = RandomForestRegressor()

In [19]:
Boosting_mod.fit(x_train,y_train)
Forest_mod.fit(x_train,y_train)

C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
print(Boosting_mod.score(x_test,y_test))
print(Forest_mod.score(x_test,y_test))

0.8495203770608619
0.8591908047601974


In [21]:
def print_metrics(y_true, y_predicted, n_parameters):
    ## First compute R^2 and the adjusted R^2
    r2 = sklm.r2_score(y_true, y_predicted)
    r2_adj = r2 - (n_parameters - 1)/(y_true.shape[0] - n_parameters) * (1 - r2)
    
    ## Print the usual metrics and the R^2 values
    print('Mean Square Error      = ' + str(sklm.mean_squared_error(y_true, y_predicted)))
    print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(y_true, y_predicted))))
    print('Mean Absolute Error    = ' + str(sklm.mean_absolute_error(y_true, y_predicted)))
    print('Median Absolute Error  = ' + str(sklm.median_absolute_error(y_true, y_predicted)))
    print('R^2                    = ' + str(r2))
    print('Adjusted R^2           = ' + str(r2_adj))
   

y_Boosting = Boosting_mod.predict(x_test)
y_Forest = Forest_mod.predict(x_test)

   

In [22]:
print_metrics(y_test, y_Boosting, 28)
print_metrics(y_test, y_Forest, 28)

Mean Square Error      = 0.7708205448359451
Root Mean Square Error = 0.8779638630581245
Mean Absolute Error    = 0.6416124553752368
Median Absolute Error  = 0.47777104232372025
R^2                    = 0.8495203770608619
Adjusted R^2           = 0.8428488666842499
Mean Square Error      = 0.7212845066508069
Root Mean Square Error = 0.8492847029417208
Mean Absolute Error    = 0.6081633612470605
Median Absolute Error  = 0.42928332449449425
R^2                    = 0.8591908047601974
Adjusted R^2           = 0.8529480325574476


In [23]:
# cross validation
Results_Boosting = cross_validate(Boosting_mod,x,y,scoring="r2",cv=5)
Results_Forest = cross_validate(Forest_mod,x,y,scoring="r2",cv=5)


In [24]:
Boosting_test_scores = Results_Boosting['test_score']
Boosting_train_scores = Results_Boosting['train_score']
Forest_test_scores = Results_Forest['test_score']
Forest_train_scores = Results_Forest['train_score']

print(np.mean(Boosting_train_scores))
print(np.mean(Boosting_test_scores))
print(np.mean(Forest_train_scores))
print(np.mean(Forest_test_scores))

0.9080295949183164
0.861692686997999
0.9759867505485171
0.8770043271673703


C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [25]:
nr.seed(123)
inside = ms.KFold(n_splits=5, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=5, shuffle = True)

In [26]:
nr.seed(3456)
## Define the dictionary for the grid search and the model object to search on
param_grid_Boosting = {"n_estimators": range(20,501,20)}
param_grid_Forest = {"n_estimators": range(20,501,20)}

## Perform the grid search over the parameters
Grid_Boosting = ms.GridSearchCV(estimator = Boosting_mod, param_grid = param_grid_Boosting, 
                      cv = inside, # Use the inside folds
                      scoring = 'r2',
                      return_train_score = True)

Grid_Forest = ms.GridSearchCV(estimator = Forest_mod, param_grid = param_grid_Forest, 
                      cv = inside, # Use the inside folds
                      scoring = 'r2',
                      return_train_score = True)


In [27]:
## Fit thhe cross validated grid search over the data 
Grid_Boosting.fit(x,y)
Grid_Forest.fit(x,y)



GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': range(20, 501, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [28]:
 Grid_Boosting.best_params_, Grid_Boosting.best_score_

({'n_estimators': 500}, 0.8758794120048428)

In [29]:
Grid_Forest.best_params_, Grid_Forest.best_score_

({'n_estimators': 440}, 0.8856713891453629)

In [30]:
final_results_Boosting = cross_validate(Grid_Boosting.best_estimator_,x_train,y_train)
final_results_Forest = cross_validate(Grid_Forest.best_estimator_,x_train,y_train)

C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [31]:
test_scores_Boosting = final_results_Boosting['test_score']
train_scores_Boosting = final_results_Boosting['train_score']
print(np.mean(train_scores_Boosting))
print(np.mean(test_scores_Boosting))

0.987865579882197
0.8527832861488114


C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [32]:
test_scores_Forest = final_results_Forest['test_score']
train_scores_Forest = final_results_Forest['train_score']
print(np.mean(train_scores_Forest))
print(np.mean(test_scores_Forest))

0.982048201682257
0.8660344372205239


In [33]:
estimator = GradientBoostingRegressor(learning_rate=0.1,
                                      n_estimators=500)
estimator.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [34]:
estimator.score(x_test,y_test)

0.8726334882441518

In [35]:
estimator2= RandomForestRegressor(n_estimators=440)
estimator2.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=440, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [36]:
estimator2.score(x_test,y_test)

0.8735142254144427

### prediction

In [37]:
test_values_processed=np.array(pd.read_csv('test_values_processed.csv', header=None))


In [43]:
estimator2.predict(test_values_processed)

array([6.1359712 , 4.39232925, 0.67385261, ..., 5.99436803, 5.43793471,
       5.56173857])

In [44]:
prediccion=pd.DataFrame(estimator2.predict(test_values_processed),columns=['evictions'])
prediccion.index.names=['row_id']

In [45]:
prediccion['evictions']=np.expm1(prediccion['evictions'])

In [46]:
prediccion['evictions']= prediccion['evictions'].astype(np.int64)
prediccion

,evictions
row_id,
0,461
1,79
2,0
3,8
4,198
5,324
6,1
7,50
8,1280


In [47]:
prediccion.to_csv('prediccion.csv')